# Recurrent Neural Network Regression task - Bike sharing

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return has become automatic. Through these systems, a user is able to easily rent a bike from a particular position and return it at another place.

The dataset contains the hourly count of rental bikes between years 2011 and 2012 in the Capital Bikeshare system (Wasington DC) with the corresponding weather and seasonal information.

The goal of this task is to train a regressor to predict total counts of bike rentals based on the provided features for a given hour. 

## Data source
[http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)

## Feature description
* **dteday** - date time stamot
* **season** - season (1: spring, 2: summer, 3: fall, 4: winter)
* **yr** - year (0: 2011, 1: 2012)
* **mnth** - month (1 to 12)
* **hr** - hour (0 to 23)
* **holiday** - 1 if the day is a holiday, else 0 (extracted from [holiday schedules](https://dchr.dc.gov/page/holiday-schedules))
* **weekday** - day of the week (0 to 6)
* **workingday** - is 1 if day is neither weekend nor holiday, else 0.
* **weathersit** 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
* **temp** - Normalized temperature in degrees of Celsius.
* **atemp** - Normalized feeling temperature in degrees Celsius.
* **hum** - Normalized relative humidity.
* **windspeed** - Normalized wind speed.
* **casual** - Count of casual users.
* **registered** - Count of registered users.
* **cnt** -  Count of total rental bikes including both casual and registered. This is the target value. 

In [26]:
import pandas as pd
data = pd.read_csv('../data/bikes.csv', sep=',')
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Recurrent Neural Network Regressor

Implement a recurrent neural network regressor. Sort the data by time stamp and deal with it as it was a time series. Be aware of using data from the past as test data.

In [27]:
data.sort_values(['dteday', 'hr'])
data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Add some features from the past

Add the target feature from the previous hour.

In [28]:
cnt = data['cnt']
data['hist'] = cnt.shift(1)
data = data[1:]

X_all = data[['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit','temp', 'atemp', 'hum', 'windspeed', 'hist']]
y_all = data['cnt']

data.head()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,hist
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,16.0
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,40.0
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,32.0
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,13.0
5,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1,1.0


### Sequences

Prepare train and test data sequences.

In [29]:
import numpy as np

window = 10

X_all = X_all.values
y_all = y_all.values

X_seq = []
y_seq = []
for i in range(window, len(X_all) + 1):
    X_seq.append(X_all[i-window: i])
    y_seq.append(y_all[i-1])

X_seq = np.asarray(X_seq)
y_seq = np.asarray(y_seq)

### Split the data into train and test data sets

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, 
    y_seq,
    random_state=1,
    test_size=0.1)

#split_index = int(X_seq.shape[0]*0.9)
#X_train = X_seq[:split_index,:,:]
#X_test = X_seq[split_index:,:,:]
#y_train = y_seq[:split_index]
#y_test = y_seq[split_index:]

print('Train size: {}'.format(X_train.shape))
print('Test size: {}'.format(X_test.shape))

Train size: (15632, 10, 13)
Test size: (1737, 10, 13)


Standardize the features

In [31]:
from sklearn.preprocessing import StandardScaler

train_shape = X_train.shape
test_shape = X_test.shape

scaler = StandardScaler()
scaler.fit(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2]))
X_train = scaler.transform(X_train.reshape(train_shape[0]*train_shape[1], train_shape[2])).reshape(train_shape)
X_test = scaler.transform(X_test.reshape(test_shape[0]*test_shape[1], test_shape[2])).reshape(test_shape)

### Training a regressor
Design and train a recurrent regression model with at least one [LSTM](https://keras.io/layers/recurrent/) layer. Use the [mean squared error](https://keras.io/losses/) loss function. Experiment with various architectures, [activation functions](https://keras.io/activations/).

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM

model = Sequential()

model.add(LSTM(32, input_shape=(window, 13)))
model.add(Dropout(0.15))
model.add(Dense(32))
model.add(Dense(1, activation='linear'))

Compile the model

In [33]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae'])

Train the model

In [34]:
model.fit(X_train, y_train,
          batch_size = 128, epochs = 100, verbose=1,
          validation_data=(X_test, y_test))

Train on 15632 samples, validate on 1737 samples
Epoch 1/100
15632/15632 [==============================] - 2s 106us/step - loss: 60450.1726 - mean_absolute_error: 173.5306 - val_loss: 51210.5688 - val_mean_absolute_error: 155.5466
Epoch 2/100
15632/15632 [==============================] - 1s 55us/step - loss: 35124.1264 - mean_absolute_error: 121.3019 - val_loss: 27963.7133 - val_mean_absolute_error: 102.9395
Epoch 3/100
15632/15632 [==============================] - 1s 58us/step - loss: 19678.8153 - mean_absolute_error: 86.1801 - val_loss: 17101.5530 - val_mean_absolute_error: 79.0430
Epoch 4/100
15632/15632 [==============================] - 1s 55us/step - loss: 13619.6177 - mean_absolute_error: 72.3741 - val_loss: 12825.8206 - val_mean_absolute_error: 70.4590
Epoch 5/100
15632/15632 [==============================] - 1s 58us/step - loss: 10736.8679 - mean_absolute_error: 64.5372 - val_loss: 10260.0054 - val_mean_absolute_error: 62.3274
Epoch 6/100
15632/15632 [=====================

15632/15632 [==============================] - 1s 57us/step - loss: 1529.2381 - mean_absolute_error: 25.9599 - val_loss: 1298.2100 - val_mean_absolute_error: 23.3741
Epoch 47/100
15632/15632 [==============================] - 1s 54us/step - loss: 1520.8327 - mean_absolute_error: 26.0326 - val_loss: 1140.9899 - val_mean_absolute_error: 21.9834
Epoch 48/100
15632/15632 [==============================] - 1s 55us/step - loss: 1495.8706 - mean_absolute_error: 25.6368 - val_loss: 1206.6199 - val_mean_absolute_error: 22.8114
Epoch 49/100
15632/15632 [==============================] - 1s 53us/step - loss: 1503.8010 - mean_absolute_error: 25.8031 - val_loss: 1154.0710 - val_mean_absolute_error: 22.3297
Epoch 50/100
15632/15632 [==============================] - 1s 54us/step - loss: 1480.4286 - mean_absolute_error: 25.6606 - val_loss: 1158.5566 - val_mean_absolute_error: 22.5171
Epoch 51/100
15632/15632 [==============================] - 1s 56us/step - loss: 1469.6611 - mean_absolute_error: 25.6

Epoch 92/100
15632/15632 [==============================] - 1s 59us/step - loss: 1266.2844 - mean_absolute_error: 23.9805 - val_loss: 1017.7967 - val_mean_absolute_error: 21.1718
Epoch 93/100
15632/15632 [==============================] - 1s 64us/step - loss: 1269.6764 - mean_absolute_error: 23.9089 - val_loss: 1068.4864 - val_mean_absolute_error: 21.3489
Epoch 94/100
15632/15632 [==============================] - 1s 61us/step - loss: 1219.7093 - mean_absolute_error: 23.4918 - val_loss: 1080.8889 - val_mean_absolute_error: 21.2953
Epoch 95/100
15632/15632 [==============================] - 1s 56us/step - loss: 1232.2090 - mean_absolute_error: 23.7469 - val_loss: 1025.8620 - val_mean_absolute_error: 21.1656
Epoch 96/100
15632/15632 [==============================] - 1s 56us/step - loss: 1214.2875 - mean_absolute_error: 23.5815 - val_loss: 1064.3900 - val_mean_absolute_error: 21.3167
Epoch 97/100
15632/15632 [==============================] - 1s 57us/step - loss: 1225.4857 - mean_absolut

### Evaluate the models

Measure mean squared error and mean absolute error evaluation metrics on both train and test data sets. Compute the mean and standard deviation of the target values.

In [35]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred = model.predict(X_test)
print ("Test mean: {}, std: {}".format(np.mean(y_test), np.std(y_test)))
print("Test Root mean squared error: {:.2f}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("Test Mean absolute error: {:.2f}".format(mean_absolute_error(y_test, y_pred)))

Test mean: 196.9222797927461, std: 187.62398319999474
Test Root mean squared error: 32.30
Test Mean absolute error: 21.40


In [36]:
y_pred = model.predict(X_train)
print("Train Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train Mean absolute error: %.2f"
      % mean_absolute_error(y_train, y_pred))

Train Root mean squared error: 28.66
Train Mean absolute error: 19.14
